In [ ]:
%pip install -q numpy
%pip install -q scipy
%pip install -q matplotlib
%pip install -q scikit-learn
%pip install -q pandas

In [ ]:
import numpy as np
from numpy import ndarray

import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt

import pandas as pd
from pandas import DataFrame

In [ ]:
from os import getcwd

CWD: str = getcwd()
ASSETS_DIR: str = CWD + "/Assets-2_2"
DATASET_DIR: str = ASSETS_DIR + "/heightWeight.csv"

# Questão A

In [ ]:
def linear_function(x: ndarray) -> ndarray:
    return x * 2 + 1

In [ ]:
def uniform_distribution(x: ndarray) -> ndarray:
    return np.random.uniform(size=len(x))

In [ ]:
X_POINTS: ndarray = np.linspace(0, 1, 9)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
ax1, ax2 = axes

# ---- Linear function ----
y1: ndarray = linear_function(X_POINTS)
ax1.scatter(X_POINTS, y1)

# get the linear regression
slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(X_POINTS, y1)
line1: ndarray = slope1 * X_POINTS + intercept1
ax1.plot(X_POINTS, line1, label="Linear regression", linestyle="--")

# ---- Uniform distribution ----
y2: ndarray = uniform_distribution(X_POINTS)
ax2.scatter(X_POINTS, y2)

# get the linear regression
slope2, intercept2, r_value2, p_value2, std_err2 = stats.linregress(X_POINTS, y2)
line2: ndarray = slope2 * X_POINTS + intercept2
ax2.plot(X_POINTS, line2, label="Linear regression", linestyle="--")

ax1.set_title("Linear function")
ax2.set_title("Uniform Distribution")

plt.plot()

# Questão B

In [ ]:
BINS: int = 40
DENSITY: bool = True

## Carregando o dataset de pesos e alturas do [Kaggle](https://www.kaggle.com/datasets/burnoutminer/heights-and-weights-dataset?resource=download)

Esse dataset contém o total de 25 mil amostras de diferentes humanos de 18 anos de idades. Suas colunas são `Height(Inches)` em polegadas (_inches_) e `Weight(Pounds)` em libras (_pounds_). Utilizaremos apenas a coluna de altura (`Height(Inches)`) em passaremos ela para centímetros para melhorar a visualização

In [ ]:
df: DataFrame = pd.read_csv(DATASET_DIR)

display(df)

## Manipulação dos dados

Iremos transformar a coluna `Height(Inches)` em valores mais comuns para o brasileiro, então converteremos os valores para centímetros e iremos trocar o nome da coluna.

Para conversão, basta multiplicar a quantidade de polegadas vezes 2.54.

In [ ]:
TARGET_COLUMN: str = "height"

# create a copy, rename the Height(Inches) to height, return just the height column
df_cleaned: DataFrame = df.copy().rename(columns={"Height(Inches)":TARGET_COLUMN}).drop(columns=["Index", "Weight(Pounds)"])

# convert to cm
df_cleaned[TARGET_COLUMN] = df_cleaned[TARGET_COLUMN].map(lambda x: x*2.54)

display(df_cleaned)

## Análise da altura

In [ ]:
mean: float = df_cleaned[TARGET_COLUMN].mean()
std: float = df_cleaned[TARGET_COLUMN].std() # type: ignore

confidences: tuple[float, ...] = (0.85, 0.9, 0.95, 0.99)
results: dict[float, tuple] = {}

for confidence in confidences:
    interval: tuple = stats.norm.interval(confidence, loc=mean, scale=std)
    print(f"Para uma confiança de {int(confidence * 100)}%, temos o intervalo: [{interval[0]}, {interval[1]}]")
    results[confidence] = interval

## Criando o histograma para mostrar graficamente a distribuição

In [ ]:
colors: tuple[str, ...] = ('#66b3ff', '#3399ff', '#0066cc', '#0044cc')

plt.figure(figsize=(10, 6))

# histogram
plt.hist(df_cleaned[TARGET_COLUMN], density=DENSITY, bins=BINS, alpha=0.3, color='grey', label="Dados (Amostra)")

# mean line
plt.axvline(mean, color="red", linestyle="--", label=f"Média: {mean:.5f}")

# confidence bar
y_max: float = plt.gca().get_ylim()[1] # graphic top
confidence_heights: tuple[float, float, float, float] = (y_max*0.1, y_max*0.2, y_max*0.3, y_max*0.4)

for i, (confidence, interval) in enumerate(results.items()):
    erro = (interval[1] - interval[0]) / 2

    plt.errorbar(mean, confidence_heights[i], xerr=erro, fmt="o", capsize=8, color=colors[i], label=f"Intervalo de Confiança: {int(confidence*100)}%")

plt.ylim(0, y_max * 1.2) # increase the graphic height
plt.title("Distribuição da Altura e Intervalo de Confiança")
plt.xlabel("Altura (cm)")
plt.ylabel("Densidade (número de amostras no bin)")
plt.legend()
plt.show()

# Questão C

# Questão D